---
# **FLAML API Demonstration**
---

This notebook is designed to guide you through the core API functions used in **FLAML (Fast and Lightweight AutoML)** , Microsoft’s efficient, budget-aware automated machine learning library. While the main workflow for your project may live elsewhere, this notebook serves as a **standalone, hands-on companion** that breaks down how FLAML works under the hood and how to use it effectively across different ML tasks.

Whether you're new to AutoML or simply curious about how FLAML achieves strong results with minimal computational cost, this notebook walks through the essentials. We’ll explore how FLAML **searches for models**, **manages resources**, **adapts its search space**, and **balances accuracy with efficiency**. Along the way, we’ll demonstrate practical examples for **classification**, **regression**, and **time-series forecasting** using FLAML’s simple yet powerful Python API.


---
---

## 1. **Introduction**

**FLAML (Fast and Lightweight AutoML)** is an open-source Python library developed by Microsoft Research. Its goal is to provide accurate and efficient automated machine learning without the heavy computational overhead typical of other AutoML frameworks.

Most AutoML tools are powerful yet computationally heavy. **FLAML solves this problem by offering a budget-aware, resource-optimized, and highly flexible AutoML engine.**

<br>
This notebook provides a mini-course-style walkthrough of FLAML. By the end, you will understand:

- **How FLAML automates model selection and hyperparameter optimization**  
- **The design principles that make FLAML lightweight and efficient**  
- **The internal logic** (search spaces, pruning, and cost-aware optimization)  
- **How to interpret FLAML’s outputs and optimization strategy**  
- **Demonstrations** of FLAML applied to classification, regression, and time series tasks  
- **When FLAML is a good fit—and when manual tuning may be preferable**  
- **How to integrate FLAML into a larger ML workflow or pipeline**

If you're exploring AutoML for the first time, don’t worry—each step is explained clearly with practical examples, making this notebook both a **learning resource** and a **practical reference** for real-world ML projects.


---
## 2. **Installation and Setup**

Before getting started, install the core libraries used in this notebook.
FLAML integrates smoothly with common Python machine learning tools such as scikit-learn, pandas, and matplotlib.

#### Option 1: Using Jupyter Notebook

In [1]:
!pip install flaml scikit-learn pandas matplotlib

#### Option 2: Using a Terminal or Command Prompt

In [2]:
# pip install flaml scikit-learn pandas matplotlib

---
---
## 3. **Understanding FLAML: What It Is and What It Does ?!**

FLAML automates the most time-consuming parts of the machine learning workflow—model selection, hyperparameter tuning, and optimization under constraints. 

Its design focuses on **efficiency, speed, and low computational cost**, making it ideal for both research and production environments.


---

### 3.1 **Problems FLAML Solves**

Traditional ML workflows often require:

- Deep knowledge of many model families  
- Expertise in configuring complex hyperparameters  
- Repeated cycles of trial-and-error  
- Large amounts of experimentation time  

FLAML minimizes these requirements by making intelligent, cost-aware decisions that reduce training time while maintaining strong performance.


---

### 3.2 **Core Capabilities**

#### **✔ Model Selection**

FLAML automatically considers and evaluates multiple algorithm families, such as:

> LightGBM <br>
> XGBoost <br>
> Random Forest / Extra Trees <br>
> Linear models <br>
> Specialized forecasting models <br>

It dynamically decides **which models to try and when to stop**, based on performance and cost signals.

#### **✔ Hyperparameter Optimization**

FLAML performs **cost-aware hyperparameter search**, optimizing for:

> Expected performance <br> 
> Training cost  <br>
> Model complexity  <br>
> Inference time  <br>

This avoids expensive grid/random searches and focuses computation where it matters most.


#### **✔ Budget-Constrained Optimization**

Users can enforce strict resource constraints, including:

> Time budget  <br>
> Memory budget <br>
> Maximum number of trials<br>
> Evaluation metric (accuracy, AUC, MAE, F1, etc.)  <br>

FLAML seeks the best possible model **within those limits**, rather than trying to maximize performance at all costs.


#### **✔ Supported Tasks**

FLAML works seamlessly across diverse ML problems:

> **Classification**  <br>
> **Regression**  <br>
> **Ranking**  <br>
> **NLP tasks** (with transformers)  <br>
> **Time-series forecasting**  <br>

Its flexibility makes it suitable for both structured data and modern deep-learning workflows.


#### **✔ Flexible Configuration Options**

Advanced users can customize FLAML extensively:

> **Estimator list**: restrict or curate which algorithms FLAML tries <br> 
> **Search space**: define or override hyperparameter ranges  <br>
> **Custom metrics**: optimize for domain-specific objectives  <br>
> **Custom learners**: integrate your own models into FLAML’s search engine <br> 

These options let FLAML function as a fully tunable component of larger pipelines.


---

### 3.3 **Why FLAML Matters?**

FLAML is designed to be:

- **Lightweight** — minimal overhead compared to other AutoML frameworks  
- **Fast** — often outperforms Optuna, Hyperopt, and Ray Tune in speed  
- **Resource-efficient** — runs well on CPUs and low-cost environments  
- **Interpretable** — clear logs, trial histories, and model summaries  
- **Pipeline-friendly** — easy integration with scikit-learn and custom workflows  

These qualities make FLAML a practical choice for **real-world, budget-conscious machine learning**, from experiments to deployment.


---
### 3.4 **Understanding FLAML Outputs**
FLAML provides several useful attributes after training:

- **`best_estimator`** – the algorithm FLAML found to perform best.  
- **`best_config`** – the hyperparameter configuration of the best model.  
- **`best_loss`** – the lowest achieved value of the metric (for classification accuracy, this is 1 - accuracy).  
- **`best_model`** – the trained model object (e.g., a scikit-learn estimator).  
- **`leaderboard`** – a pandas DataFrame of all tried models and their scores.  
- **`time_taken`** – total time used in the search.  
- **`search_state`** – internal details of the search process (advanced usage).  

These outputs help us inspect and understand what FLAML did under the hood.

---
---
## 4. **Example 1 : Task - Classification** (Iris Dataset)

The Iris dataset is a classic dataset in machine learning used for **classification** tasks. In this example, we will use FLAML to automatically find a good model for classifying Iris flowers, all within a short time budget.

### What You’ll Learn Here
- How to load a dataset and split it into training and testing sets.  
- How to set up a FLAML AutoML instance with a time budget and metric.  
- How to train and tune models automatically using FLAML.  
- How to evaluate the best model and interpret its outputs.

### Steps:
1. Load the Iris data and split into train/test.
2. Create an `AutoML` instance and specify a time budget and metric.
3. Call `fit()` to let FLAML train and tune models.
4. Evaluate the best model on the test set.

---
### Code Example: Using FLAML for Classification

In [3]:
# Import Relevant Libraries
import warnings
from flaml import AutoML
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Suppress sklearn warnings (optionally)
warnings.filterwarnings("ignore")

# Load dataset and split
X_clf, y_clf = load_iris(return_X_y=True)
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42
)

# Set up AutoML with a time budget (seconds) and optimization metric
automl = AutoML(log_file_name="", verbose=0)  # Disable logging and info messages (optionally)

automl_settings = {
    "time_budget": 20,            # seconds
    "metric": "accuracy",         # optimize for accuracy
    "task": "classification",     # task type
    # "estimator_list": ["lgbm", "rf", "xgboost"],  # optional: restrict to specific models
}

# Fit the model (log_file_name="" disables logging for this run too)
automl.fit(X_train=X_train_clf, y_train=y_train_clf, log_file_name="", **automl_settings)

# Predictions on test
preds = automl.predict(X_test_clf)

# Outputs
print("\nBest estimator:", automl.best_estimator)
print("Best hyperparameters:", automl.best_config)
print("Best loss (metric):", automl.best_loss)
print("Test accuracy:", accuracy_score(y_test_clf, preds))


Best estimator: lrl1
Best hyperparameters: {'C': np.float64(1.0)}
Best loss (metric): 0.025
Test accuracy: 1.0


---
### Output Explanation

- **Best estimator:** The model FLAML found to perform best on the training data (here, logistic regression with L1 regularization).
- **Best hyperparameters:** The optimal hyperparameter configuration for the chosen model. In this case, the regularization strength `C` is 1.0.
- **Best loss (metric):** The value of the loss function corresponding to the best model. Lower values indicate better performance.
- **Test accuracy:** How well the selected model performs on unseen test data. A value of 1.0 indicates perfect classification for this split.

---
### Insights for Beginners

- **Why use FLAML for classification?** FLAML automatically selects the best classification algorithm (like logistic regression, random forest, or XGBoost) and tunes hyperparameters, saving you from manually testing multiple models.
- **Time budget matters:** With a short time budget, FLAML finds reasonably good classifiers quickly. Increasing the budget can allow FLAML to explore more models and hyperparameter combinations, potentially improving accuracy.
- **Understanding model choices:** For classification tasks, FLAML may choose simple linear models (like logistic regression) for small datasets or tree-based models (like random forest or XGBoost) for more complex patterns. Reviewing `automl.best_estimator` helps you understand which type of model suits your data.
- **Evaluating predictions:** Always check metrics like accuracy, precision, recall, or F1-score depending on your classification problem. FLAML optimizes a chosen metric (`accuracy` here), but you can adapt it for imbalanced datasets.
- **Practical tip:** Start with small classification datasets like Iris to get familiar with FLAML’s workflow. Observe which models and hyperparameters are selected to gain intuition for more complex classification tasks.

---
---
## 5. **Example 2 : Task - Regression** (California Housing Dataset)

The California Housing dataset is a widely used dataset for regression tasks, where the goal is to predict house prices based on various features. In this example, we will use FLAML to automatically select and tune a regression model within a short time budget.

This will show how FLAML handles a larger dataset and a different metric.

### What You’ll Learn Here
- How to load a regression dataset and split it into training and testing sets.  
- How to set up a FLAML AutoML instance for regression tasks.  
- How to train and tune models automatically using FLAML.  
- How to evaluate the best model using an appropriate regression metric (RMSE).  

### Steps
1. Load the California Housing dataset and split into train/test sets.  
2. Create an AutoML instance and specify a time budget and regression metric.  
3. Call `fit()` to let FLAML train and tune models.  
4. Evaluate the best model on the test set using RMSE.


---
### Code Example: Using FLAML for Regression

In [4]:
# Import Relevant Libraries
from flaml import AutoML
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Load dataset and split
data = fetch_california_housing()
X_reg, y_reg = data.data, data.target
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Set up AutoML for regression
automl_reg = AutoML(log_file_name="", verbose=0)  # Disable logging and info messages (optionally)
automl_reg.fit(
    X_train=X_train_reg, 
    y_train=y_train_reg, 
    task="regression", 
    time_budget=25, 
    metric="rmse"
)

# Predictions and evaluation
preds_reg = automl_reg.predict(X_test_reg)
rmse = np.sqrt(mean_squared_error(y_test_reg, preds_reg))

# Outputs
print("\nBest estimator (regression):", automl_reg.best_estimator)
print("Best config (regression):", automl_reg.best_config)
print("Test RMSE:", rmse)


Best estimator (regression): xgb_limitdepth
Best config (regression): {'n_estimators': 740, 'max_depth': 9, 'min_child_weight': np.float64(24.510233543504107), 'learning_rate': np.float64(0.04052344076603214), 'subsample': np.float64(0.8809985595678514), 'colsample_bylevel': np.float64(0.8490633393816673), 'colsample_bytree': np.float64(0.8053272322153723), 'reg_alpha': np.float64(0.006254798766353445), 'reg_lambda': np.float64(0.0887439544402997)}
Test RMSE: 0.43432197974277365


---
### Output Explanation

- **Best estimator:** `xgb_limitdepth` — FLAML selected this XGBoost variant with a depth limit as the most effective model for this regression task.  
- **Best config:** The optimal hyperparameters found by FLAML, including number of trees (`n_estimators`), tree depth (`max_depth`), learning rate, subsampling ratios, column sampling ratios, and regularization parameters (`reg_alpha` and `reg_lambda`). These settings define the model structure and training behavior.  
- **Test RMSE:** `0.4489` — The Root Mean Squared Error on the test set, indicating how far the predicted house prices are from the actual values on average. Lower RMSE means better prediction accuracy.

### Insights for Beginners

- **Why FLAML picked `xgb_limitdepth`:** For this dataset, a tree-based model with depth constraints balances bias and variance well, which is often effective for regression tasks with non-linear feature relationships.  
- **Hyperparameter impact:** Parameters like `max_depth` and `min_child_weight` control tree complexity, while `learning_rate` and `n_estimators` affect training dynamics. Subsampling (`subsample`, `colsample_bytree`) helps prevent overfitting.  
- **Interpreting RMSE:** An RMSE of ~0.45 means that, on average, predicted house prices differ from actual prices by roughly 0.45 units (in normalized scale). Use domain knowledge to assess if this is acceptable.  
- **Practical tip:** Experimenting with time budgets and allowed models can further improve performance. Reviewing `best_config` helps you understand which hyperparameters FLAML considers optimal for your data.

---
---
## 6. **Example 3 : Task - Time Series Forecasting** (Synthetic Dataset)

FLAML also supports time series forecasting tasks, handling:
- Rolling or expanding windows for training  
- Seasonal periods and lags (with the `period` parameter)  
- Selecting appropriate models for forecasting (e.g., Prophet, ARIMA, or tree-based models)  

In this example, we'll create a synthetic univariate daily time series with seasonality and noise, and use FLAML to forecast future values.

### What You’ll Learn Here
- How to generate a synthetic time series with seasonality and noise.  
- How to split the data into training and testing sets.  
- How to set up FLAML for time series forecasting.  
- How to forecast future values and evaluate results.  

### Steps
1. Create a synthetic daily time series with a sine wave and added noise.  
2. Split the data into training and test sets.  
3. Set up a FLAML AutoML instance for time series forecasting with a specified `period`.  
4. Train the model and forecast future values.  
5. Inspect the first few predicted values.  

---

### Code Example: Using FLAML for Time Series Forecasting

In [8]:
# Import Relevant Libraries
import logging
import warnings
import numpy as np
import pandas as pd
from flaml import AutoML

# Suppress sklearn warnings (optionally)
warnings.filterwarnings("ignore")

# Suppress all logging (optionally)
logging.disable(logging.INFO)

# Create a synthetic time series (sine wave with noise)
dates = pd.date_range("2020-01-01", periods=200, freq="D")
values = np.sin(np.arange(200) / 10) + np.random.normal(0, 0.2, 200)
data_ts = pd.DataFrame({"date": dates, "value": values})

# Split into train and test
train_ts = data_ts.iloc[:150]
test_ts = data_ts.iloc[150:]

# Set up AutoML for forecasting
automl_ts = AutoML(log_file_name="", verbose=0)  # Disable logging and info messages (optionally)
automl_ts.fit(
    X_train=train_ts[["date"]],   # dates as features
    y_train=train_ts["value"],    # values to forecast
    task="ts_forecast",
    time_budget=30,
    period=7,                     # assuming weekly seasonality for daily data
)

# Forecast on the test dates
forecast_values = automl_ts.predict(test_ts[["date"]])
print("\nForecast for first 5 test dates:", forecast_values[:5], sep="\n")


Forecast for first 5 test dates:
150    0.488949
151    0.364290
152    0.222492
153    0.056701
154    0.018077
Name: value, dtype: float64


---
### Output Explanation

- **Forecast values:** Predicted values for the test period. For example, the first few forecasts may look like 0.49, 0.36, 0.22, 0.06, 0.02 (approximate values). Note that these numbers will **change slightly with each run** because the synthetic data includes random noise.  
- **Period parameter:** The `period=7` indicates weekly seasonality, which helps FLAML consider repeating patterns in the data.  
- **Metric:** By default, FLAML uses MAPE (Mean Absolute Percentage Error) for time series tasks.  
- **Model choice:** FLAML automatically selects suitable forecasting models (Prophet, ARIMA, LightGBM, etc.) based on availability, the dataset, and the time budget.

---
### Insights for Beginners

- **Why FLAML for forecasting?** It automates both model selection and hyperparameter tuning, reducing manual effort and experimentation in time series tasks.  
- **Importance of period:** Providing a seasonal period helps FLAML capture regular patterns and improves forecast accuracy.  
- **Synthetic data:** Since we added random noise to the sine wave, the forecasts will vary slightly with each run. This is expected and simulates real-world variability.  
- **Practical tip:** Visualize the forecasts against actual values to understand trends and deviations. Adjust `period` or increase `time_budget` for potentially better predictions.  

**Next steps:** You can save the trained AutoML object for reuse, allowing you to forecast new time periods without retraining. See the next section for an example.

---
---
## 7. **Model Persistence: Saving and Loading AutoML Models**

After training a model with FLAML, you might want to **reuse it later without retraining**. FLAML allows you to save the **entire AutoML object**, which includes:

- The best model (`best_model`)
- Hyperparameters (`best_config`)
- All search results and internal state

This makes it easy to **persist your work** and quickly reload it for future predictions across various tasks like **classification, regression, and time series tasks**.


---
### Code Example: To Save and Load Classification Model

In [6]:
# Import Relevant Libraries
import pickle
from sklearn.metrics import accuracy_score

# Save the AutoML object
with open("automl_clf.pkl", "wb") as f:
    pickle.dump(automl, f)

# Load the AutoML object back
with open("automl_clf.pkl", "rb") as f:
    automl_clf_loaded = pickle.load(f)

# Verify the loaded model works
clf_preds = automl_clf_loaded.predict(X_test_clf)
print("\nLoaded classification model test accuracy:", accuracy_score(y_test_clf, clf_preds))


Loaded classification model test accuracy: 1.0


**Expected Output (classification):**  
The test accuracy should be close to the original model’s accuracy (e.g., 1.0 for the Iris dataset), but might slightly vary on different runs if the dataset is small or models have randomness.

---
### Output Explanation

- **Pickle for persistence:** Python’s `pickle` library serializes the AutoML object to disk.  
- **Includes everything:** Saving preserves the trained model, best hyperparameters, and FLAML search history.  
- **Reload and reuse:** You can load the object anytime to make predictions without retraining.  
- **Verify outputs:** Always check predictions to ensure the loaded model works as expected.  

---
### Insights for Beginners

- **Why save the AutoML object?** Retraining can take time, especially for large datasets or complex models. Saving lets you reuse the model instantly.  
- **Best practice:** Include versioning or date in filenames (e.g., `automl_clf_2025.pkl`) to track experiments.  
- **Other tasks:** The same approach works for **regression** or **time series forecasting** models—just use the respective trained AutoML object.  
- **Caution:** Pickle is Python-specific and not safe for untrusted sources. Only load objects you trust.  
- **Small dataset note:** For small datasets like Iris, results are usually stable, but slight variations can occur due to random initialization or internal model randomness.

---
---
## 8. **Integrating FLAML into a Machine Learning Pipeline**

FLAML can be embedded inside a **scikit-learn pipeline**, just like any other estimator. This allows you to combine FLAML's **model selection** with preprocessing or feature engineering steps, ensuring transformations are applied consistently during training and prediction.

---
### Code Example: Regression Pipeline

In [9]:
# Import Relevant Libraries
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from flaml import AutoML
from sklearn.metrics import mean_squared_error
import numpy as np

# Create a pipeline with a scaler and AutoML
reg_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("automl", AutoML(log_file_name="", verbose=0))
])

# Fit the pipeline on regression data
reg_pipeline.fit(
    X_train_reg, 
    y_train_reg, 
    automl__task="regression", 
    automl__time_budget=15, 
    automl__metric="rmse"
)

# Predict and evaluate
reg_preds = reg_pipeline.predict(X_test_reg)
print("\nPipeline regression model test RMSE:", np.sqrt(mean_squared_error(y_test_reg, reg_preds)))


Pipeline regression model test RMSE: 0.4576929514126948


**Expected Output (regression):**  
The RMSE should be close to the value obtained when running AutoML directly (e.g., ~0.45), but it may slightly vary due to internal randomness in the AutoML search.

---
### Notes / Insights

- **Step parameter syntax:** Use `stepName__param` (e.g., `automl__time_budget`) to pass arguments to the AutoML step in a pipeline.
- **Consistency:** Any preprocessing (like scaling) is applied automatically to both training and prediction data.
- **Task flexibility:** You can use the same pipeline structure for **classification**, **regression**, and **time series forecasting**.
- **Randomness note:** RMSE or predictions may vary slightly between runs due to randomness in model initialization or search.
- **Practical tip:** Pipelines are ideal for end-to-end workflows, combining feature transformations, AutoML search, and predictions in a single object.

---
---
## 9. **Best Practices & Recommendations**

**1. FLAML’s strength is *cost-aware* search:** Unlike brute-force hyperparameter tuning, FLAML intelligently balances performance gain against computation cost. This makes the search extremely efficient, often finding good models faster and with less resources.

**2. Great for quick prototyping:** FLAML is ideal when you need a strong baseline or a quick solution. It removes a lot of manual tuning, allowing you to get reasonable models with minimal effort.

**3. Not a drop-in replacement for expert tuning:** For mission-critical applications or when chasing the last bit of performance, human expertise in feature engineering and tuning can still outperform AutoML. FLAML gives you a strong start, but domain knowledge remains important.

**4. Ideal use cases:** FLAML shines for students and practitioners who want to accelerate experiments, for automated ML in pipelines or MLOps, and for problems where time or compute is limited.

**5. Continued learning:** Using FLAML is also a learning opportunity. By inspecting `best_config` and `best_estimator`, you can glean which models and hyperparameters work well for your data, informing your future manual model tuning.


---
## 10. **Summary and Learning Outcomes**

In this notebook, we introduced FLAML and demonstrated how it automates the machine learning workflow:

* **AutoML concept:** FLAML selects models and tunes hyperparameters using a cost-aware approach.
* **Hands-on examples:** We applied FLAML to classification (Iris dataset), regression (California Housing), and time series forecasting tasks, observing how it finds effective solutions with minimal guidance.
* **Integration:** FLAML can be seamlessly embedded in pipelines with preprocessing or feature engineering steps, making it suitable for end-to-end ML workflows.
* **Insights:** We discussed when FLAML is most useful and where manual expertise remains valuable.

FLAML brings efficient AutoML to everyone. It is a powerful assistant for building models quickly and can save significant time in both learning environments and real-world applications. By leveraging FLAML, you can focus more on understanding the problem and the data, and less on the tedious tuning of models.

---
---